<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet201(reran).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 173 kB 76.9 MB/s 
     |████████████████████████████████| 182 kB 78.0 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 77.0 MB/s 
     |████████████████████████████████| 168 kB 65.9 MB/s 
     |████████████████████████████████| 166 kB 85.0 MB/s 
     |████████████████████████████████| 166 kB 75.6 MB/s 
     |████████████████████████████████| 162 kB 82.5 MB/s 
     |████████████████████████████████| 162 kB 82.8 MB/s 
     |████████████████████████████████| 158 kB 79.0 MB/s 
     |████████████████████████████████| 157 kB 76.7 MB/s 
     |████████████████████████████████| 157 kB 77.5 MB/s 
     |████████████████████████████████| 157 kB 35.7 MB/s 
     |████████████████████████████████| 157 kB 84.7 MB/s 
     |████████████████████████████████| 157 kB 84.2 MB/s 
     |████████████████████████████████| 157 kB 79.7 MB/s 
     |███████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
 densenet201 = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_densenet201=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet201,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
densenet201.trainable=False
model_densenet201.summary()

74836368/74836368 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 densenet201 (Functional)    (None, 1920)              18321984  
                                                                 
 dropout (Dropout)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 5)                 9605      
                                                                 
Total params: 18,331,589
Trainable params: 9,605
Non-trainable params: 18,321,984
_________________________________________________________________


In [7]:
model_densenet201.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Densenet201"
      )

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_densenet201.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback(compute_flops = True)]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.2297 - accuracy: 0.4848

wandb: Adding directory to artifact (/content/wandb/run-20221216_165004-2ihomuzh/files/model-best)... Done. 0.7s


19/19 [==============================] - 178s 7s/step - loss: 1.2297 - accuracy: 0.4848 - val_loss: 0.7024 - val_accuracy: 0.7095
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.5336 - accuracy: 0.8547

wandb: Adding directory to artifact (/content/wandb/run-20221216_165004-2ihomuzh/files/model-best)... Done. 0.7s


...

19/19 [==============================] - 74s 4s/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0655 - val_accuracy: 0.9932
Epoch 32/40
19/19 [==============================] - 5s 236ms/step - loss: 0.0233 - accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9932
Epoch 33/40
19/19 [==============================] - 5s 236ms/step - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.0695 - val_accuracy: 0.9932
Epoch 34/40
19/19 [==============================] - 5s 236ms/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 0.0661 - val_accuracy: 0.9932
Epoch 35/40
19/19 [==============================] - 5s 236ms/step - loss: 0.0177 - accuracy: 0.9983 - val_loss: 0.0669 - val_accuracy: 0.9932
Epoch 36/40
19/19 [==============================] - 5s 238ms/step - loss: 0.0182 - accuracy: 0.9983 - val_loss: 0.0662 - val_accuracy: 0.9932
Epoch 37/40
19/19 [==============================] - ETA: 0s - loss: 0.0175 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221216_165004-2ihomuzh/files/model-best)... Done. 0.7s


19/19 [==============================] - 76s 4s/step - loss: 0.0175 - accuracy: 1.0000 - val_loss: 0.0628 - val_accuracy: 0.9932
Epoch 38/40
19/19 [==============================] - 5s 236ms/step - loss: 0.0146 - accuracy: 1.0000 - val_loss: 0.0632 - val_accuracy: 0.9932
Epoch 39/40
19/19 [==============================] - ETA: 0s - loss: 0.0152 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221216_165004-2ihomuzh/files/model-best)... Done. 0.5s


19/19 [==============================] - 75s 4s/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.0611 - val_accuracy: 0.9932
Epoch 40/40
19/19 [==============================] - 5s 240ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9932
CPU times: user 34min 10s, sys: 1min 6s, total: 35min 16s
Wall time: 39min 52s


In [10]:
wandb.alert(title = 'Run finished',
            text = 'densenet201')

In [11]:
wandb.finish()

accuracy,▁▆▇▇████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▇▇█████████████████████████████████████
val_loss,█▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,4.29087
accuracy,1.0
best_epoch,38
best_val_loss,0.06112
epoch,39
loss,0.01545


In [12]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
